# Spark Assginment 2: Tweets Analysis

** In this assignment, you will learn: **
* Parse Tweets from josn file
* Collect tweets from Spark SQL and DataFrames
* Sample tweets and call Waston API to do analysis
* Virtualize the results
-----
<font color='red'>IMPORTANT: </font>This notebook contains credential information, which should be private and confidential. Plase only use it for assignment. Do not share with others and out of course life cycle. 

## Part1: Parse Tweets from josn file

When crawled tweets from twitter, all the tweets information are stored in json format. SparkSQL can parse json file and transform into data frame. 

In [ ]:
import requests, StringIO,json

In [ ]:
credentials = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_9de0bf59_5fe2_4f11_8d80_ac12b45f3fca',
  'project_id':'3fd339f1bd7b4ff4a1eb8da7659e39ba',
  'region':'dallas',
  'user_id':'46476b565b864525a6a365d41fe7a280',
  'domain_id':'29cedac5a5ca492c83b9c6f324f7229d',
  'domain_name':'913169',
  'username':'Admin_28154f3d777dc89d3deb5a8285de9513c30049b3',
  'password':"""HQ.7BPLU5W1=5Yob""",
  'container':'notebooks',
  'tenantId':''
}

In [ ]:
def set_hadoop_config(credentials):
    prefix = "fs.swift.service." + credentials['name'] 
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

In [ ]:
credentials['name'] = 'keystone'
set_hadoop_config(credentials)

In [ ]:
tweets = sqlContext.read.json("swift://notebooks.keystone/Tweets.txt")

Here, we will do some simple text cleaning, including removing symbols, trimming and lowering the letters.

In [ ]:
from pyspark.sql.functions import regexp_replace, trim, col, lower
def removePunctuation(column):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        column (Column): A Column containing a sentence.

    Returns:
        Column: A Column named 'sentence' with clean-up operations applied.
    """
    return regexp_replace(trim(lower(column)),'[^a-z0-9 ]','').alias('value')

After text cleaning, we can show some result using Spark SQL

In [ ]:
tweet_text = tweets.select(removePunctuation(col('text')))
tweet_text.show(n=10)

## Colloect tweets from Spark SQL and DataFrames

Now we collect the SQL result from Spark to local and do further analysis.

In [ ]:
texts = tweet_text.sample(False,0.1).collect()
texts = [i[0] for i in texts]

You can check the texts, all texts are stored in the Python list

In [ ]:
texts[0]

## Using Wastion Tone Analyzer API to do text analysis

Before we use API, we need to install it. For **Tone Analyzer** API reference, click [Here](http://www.ibm.com/watson/developercloud/doc/tone-analyzer/)

In [ ]:
!pip install watson-developer-cloud

In [ ]:
from watson_developer_cloud import ToneAnalyzerV3

def tone(te):
    tone_analyzer = ToneAnalyzerV3(
        password =  "FwdYI5O0TaGY",
        username =  "be8de89c-f720-43eb-beec-9f52093bdb6a",
        version ='2016-05-19')
    try:
        toneresult = tone_analyzer.tone(text=te)['document_tone']['tone_categories']
        print "API success"
        return toneresult
    except:
        print "API calls reaches limitation, place try again later"
    

Call the API

In [ ]:
result = tone('\n'.join(texts))

Fetch the necessary results

In [ ]:
emotion_tone = dict([(i.values()[0],i.values()[1]) for i in result[0]['tones']])
social_tone = dict([(i.values()[0],i.values()[1]) for i in result[2]['tones']])

## Virtualize the results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
plt.barh(range(len(emotion_tone)),emotion_tone.values(),align='center',alpha=0.5,color='g')
plt.yticks(range(len(emotion_tone)),emotion_tone.keys())
plt.title('Emotion Tone')

plt.show()

In [ ]:
plt.barh(range(len(social_tone)),social_tone.values(),align='center',alpha=0.5,color='b')
plt.yticks(range(len(social_tone)),social_tone.keys())

plt.title('Social Tone')
for i, v in enumerate(social_tone.values()):
    plt.text(v/2, i-0.1, str(v), color='w')  
plt.show()

# End of Assignment
Copyright &copy; 2016 by Tianyang Zhang